# Day 1 Code Compiled

This file contains all of the Day 1 code re-written and documented to build a simple text summariser with GPT Mini.

In [1]:
import os
import requests
from dotenv import load_dotenv # loads environment variables from a .env file into your Python environment.
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# Loading environment variable

Load an the API key (environment variable) from .env into Python environment and store it as a variable.

In [ ]:
# load in the environment variables in your `.env` file
load_dotenv(override=True)

# retrieve the environment variable -> string of the API key
api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
# check the API key is valid
if not api_key:
    print("NO API key was found")
elif not api_key.startswith("sk-proj-"):
    print("Check you're using the right key")
elif api_key.strip() != api_key:
    print("The API key may have space of tab characters at the start/end")
else:
    print("API key was found!")

# 

API key was found!


# Connecting to OpenAI

Make a connection to OpenAI API Service to make a call to GPT (Frontier Model) to ask it questions

In [8]:
# Instantiate an OpenAI object
openai = OpenAI()

# Making a quick call to a Froniter Model

First, we will make a quick call to gpt-4o-mini, a smaller, more cost-effective version of OpenAI's GPT-4o model, designed to balance performance with affordability.

In [ ]:
# input message
message = "Hello, GPT! This is my first ever message to you! Hi!"

# response from GPT
response = openai.chat.completions.create(model = "gpt-4o-mini",
                                          messages=[{
                                              "role":"user",
                                              "content":message
                                          }])

print(response)

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Hello! Welcome! I'm glad you're here. How can I assist you today?", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))]


In [18]:
# printing response in a cleaner format:
print(response.choices[0].message.content)

Hello! Welcome! I'm glad you're here. How can I assist you today?


# Project 1: Summarising webpages

## Defining class to scrape text content off a webpage

In [ ]:
# some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

# a class that represents a webpage
class Website:
    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers) # sends https request to get data
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found" # define webpage title
        
        # finds all <script>, <style>, <img>, and <input> elements inside the <body> tag and removes them
        # we only want the main text
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip = True) # define webpage text


In [ ]:
# lets explore the text and title attributes stored in this class object Website
[attr for attr in dir(Website("https://edwarddonner.com")) if not attr.startswith("__")]

['text', 'title', 'url']

In [33]:
# lets try out the webpage object that gets the text off a webpage
print(Website("https://edwarddonner.com").title)
print(Website("https://edwarddonner.com").text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

## Types of Prompts:

These Frontier models expect two types of instructions:

**1. System Prompt:**
* Purpose: Provides essential context or instructions to the model about how it should behave as an assistant. E.G: Explain the role of the model.

* Example: "You are a helpful assistant designed to provide clear, concise answers. You can help with coding, answering general knowledge questions, and offering recommendations."

**2. User Prompt:**

* Purpose: What the user inputs—it's the actual conversation or query that the LLM is responding to.

* Example: "User: Can you help me write a Python function that reverses a string?"

The LLM's task is to consider the context of the system prompt while responding to the user’s input in a way that fits the given instructions.

## Defining our user prompt

In [35]:
# define a function that writes a user prompt that asks for summaries of websites
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\n\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [ ]:
# example: user prompt asking for a summary of this webpage
print(user_prompt_for(Website("https://edwarddonner.com")))

You are looking at a website titled Home - Edward Donner

The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acqu

## Messages:

The OpenAI API expects to receive messages in a particular structure. Many other APIs expect a similar structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]
```

In [43]:
# example input containing system & user prompts:
example_msg = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2+2?"}
]

# example call to OpenAI with system & user prompts:
response = openai.chat.completions.create(model="gpt-4o-mini", messages=example_msg)
print(response.choices[0].message.content)

Oh, let me dust off my math skills... It’s 4. Shocking, I know.


## Building our messsage

In [45]:
# defining our system prompt:

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."


In [53]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [54]:
messages_for(Website("https://edwarddonner.com"))

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\n\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make

## Calling the Froniter Model

We will now define a general function to call the Froniter Model to summarise the content of a webpage.

In [55]:
# call the OpenAI API
def summarise(url):
    website = Website(url) # return website object with scraped text
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website) # pass in the user prompt for a particular website
    )
    return response.choices[0].message.content # output model text

In [ ]:
# example function:
summarise("https://edwarddonner.com")

"# Summary of Edward Donner's Website\n\nThis website serves as a personal platform for Ed Donner, who is passionate about coding, experimenting with large language models (LLMs), and electronic music. He is the co-founder and CTO of Nebula.io, a company that leverages AI to enhance talent discovery and engagement. Ed has a background in leading AI startups, including one that was acquired in 2021.\n\n## Key Features:\n- **Connect Four**: A competitive platform where LLMs face off in strategic scenarios.\n- **Outsmart**: Another interactive arena for LLM competitions.\n- **About Ed**: Insights into his interests and professional journey.\n- **Posts**: A collection of resources and articles related to AI and LLMs.\n\n## Recent Announcements:\n- **January 23, 2025**: Resources for the LLM Workshop focused on hands-on experience with agents.\n- **December 21, 2024**: A warm welcome to a new audience, referred to as SuperDataScientists.\n- **November 13, 2024**: Availability of resources f

In [57]:
# A function to display this nicely in Jupyter output, using markdown
def display_summary(url):
    summary = summarise(url)
    display(Markdown(summary))

In [58]:
display_summary("https://edwarddonner.com")

# Summary of Edward Donner's Website

The website serves as a personal space for Ed Donner, a co-founder and CTO of Nebula.io, where he shares his interests in coding, LLMs (large language models), DJing, and electronic music production. Nebula.io focuses on leveraging AI to improve talent discovery and management, boasting a proprietary matching model and an award-winning platform.

## Recent Announcements
- **January 23, 2025**: LLM Workshop – Hands-on with Agents – resources available.
- **December 21, 2024**: Welcome message for SuperDataScientists.
- **November 13, 2024**: Resources for "Mastering AI and LLM Engineering" shared.
- **October 16, 2024**: Resources for transitioning from Software Engineer to AI Data Scientist released.

The site encourages connections, inviting visitors to reach out via email and social media.

# Defining this as a single class

This makes it simpler to run through the entire process with a single object

In [68]:
class SummariseWebpage:
    def __init__(self, url):
        self.url = url
        self.headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
        self.system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

    def scrape(self):
        """
          Method to scrape text and title from given url using BeautifulSoup library
        """
        response = requests.get(self.url, headers=self.headers) # sends https request to get data
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found" # define webpage title
        
        # finds all <script>, <style>, <img>, and <input> elements inside the <body> tag and removes them
        # we only want the main text
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip = True) # define webpage text
        
    def _user_prompt_for(self):
        """
          Method to format user prompt
        """
        user_prompt = f"You are looking at a website titled {self.title}"
        user_prompt += "\n\nThe contents of this website is as follows; \
    please provide a short summary of this website in markdown. \
    If it includes news or announcements, then summarize these too.\n\n"
        user_prompt += self.text
        return user_prompt
    
    def _messages_for(self):
        """
          Method to format OpenAI message (contains user & system prompt)
        """
        return [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": self._user_prompt_for()}
        ]
    
    def summarise(self):
        """
          Method that summarises the scraped webpage
        """
        response = openai.chat.completions.create(
            model = "gpt-4o-mini",
            messages = self._messages_for() # pass in the user prompt for a particular website
        )
        return response.choices[0].message.content # output model text
        
    def display_summary(self, text):
        """
          Method to display this nicely in Jupyter output, using markdown
        """
        display(Markdown(text))

In [70]:
# intialise instance of SummariseWebpage object
summary = SummariseWebpage(url="https://edwarddonner.com")

# scrape webpage and retrieve title & text
summary.scrape()

# sumamrise webpage and display it
summary.display_summary(summary.summarise())

# Summary of Edward Donner's Website

The website is a personal platform of Ed Donner, who is a coder, DJ, and CTO of Nebula.io, a company focused on applying AI in talent discovery and management. He has a background in AI startups and emphasizes his work with large language models (LLMs).

## Notable Features
- **Professional Background**: Ed is involved in developing AI solutions and has founded an AI startup, untapt, which was acquired in 2021.
- **Interests**: Beyond coding, he enjoys DJing and electronic music, albeit at an amateur level.

## News and Announcements
- **January 23, 2025**: LLM Workshop – Hands-on with Agents – resources provided.
- **December 21, 2024**: Welcoming SuperDataScientists to the community.
- **November 13, 2024**: Resources shared for "Mastering AI and LLM Engineering."
- **October 16, 2024**: Resources for transitioning from Software Engineer to AI Data Scientist have been published. 

For further engagement, Ed encourages visitors to connect with him through various social channels and to subscribe to his newsletter.